In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#https://www.youtube.com/watch?v=ZDF2LDsiLBs

In [ ]:
import sys
print(sys.version, sys.platform, sys.executable)

In [ ]:
#Volatility is one of the most prominent terms you’ll hear on any trading floor – and for good reason. 
#In financial markets, volatility captures the amount of fluctuation in prices.
#High volatility is associated to periods of market turbulence and to large price swings,
#while low volatility describes more calm and quiet markets.

In [ ]:
import os
import random
import glob
import gc
from tqdm import tqdm
import missingno as msno

import pyarrow as pa
import pyarrow.parquet as pq

import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

import warnings
warnings.filterwarnings("ignore")

In [ ]:
sample = pd.read_csv("../input/optiver-realized-volatility-prediction/sample_submission.csv")
sample

In [ ]:
#There are only row_id and target.
#target is a volatility(degree of price fluctuation) that we must predict.
# target is the total volatility for 10 minutes

In [ ]:
test = pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
test

In [ ]:
#the row_id of the submission file is the stock_id (stock id) and time_id (time id) connected by string "-".¶

In [ ]:
book_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0")
book_testparquet

In [ ]:
book_trainparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=68")
book_trainparquet[book_trainparquet["time_id"]==5]

In [ ]:
book_trainparquet.groupby(['time_id']).size()

In [ ]:
# team 1: stock id 10,14,18, same time_id
# team 2: stock id 20,24,28,same time_id
# team 3: stock id 30,34,38, same time_id
# team 4: stock id 40,44,48, same time_id
# team 5: stock id 50,54,58, same time_id
# team 6: stock id 60,64,68, same time_id

In [ ]:
## set time id = 5

In [ ]:
trade_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0")
trade_testparquet

In [ ]:
trade_trainparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=68")
trade_trainparquet[trade_trainparquet["time_id"]==5]

In [ ]:
# order book
#https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train.head()

In [ ]:
train.tail()

In [ ]:
train.shape

In [ ]:
train.dtypes

In [ ]:
train.isnull().any(axis=0) # no missing values

In [ ]:
len(set(train['stock_id'])) # 112 stock

In [ ]:
train.groupby(['stock_id']).size()

In [ ]:
len(set(train['time_id'])) # 3830

In [ ]:
#stock_id - ID code for the stock.
#time_id - ID code for the time bucket. Time IDs are not necessarily sequential but are consistent across all stocks.
#target - The realized volatility computed over the 10 minute window following the feature data 
#under the same stock/time_id. There is no overlap between feature and target data. 

In [ ]:
stock = train.groupby("stock_id")["target"].agg(["mean","median","std","count","sum"]).reset_index()
stock

In [ ]:
import seaborn as sns
plt.figure(figsize=(10, 5))
sns.histplot(train['target'], label = 'Target distribution')

In [ ]:
print("mean value=" ,stock["mean"].mean())
plt.hist(stock["mean"])

In [ ]:
#The average value is 0.003, which is close to 0.

In [ ]:
print("sum value=" ,stock["sum"].mean())
plt.hist(stock["sum"])

In [ ]:
#The total volatility during this period is 14.8, max, and there are more than 30.

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(train['time_id'], label = 'Time_id distribution')

In [ ]:
plt.figure(figsize=(10, 5))
sns.scatterplot(data=train, x="time_id", y="target")

In [ ]:
#Taking the first row of data, it implies that the realized vol of the target bucket for time_id 5, stock_id 0 is 0.004136

In [ ]:
##
stock68 = train[train["stock_id"]==68].reset_index()
min_index_68 = stock68["target"].idxmin()
min_time_id_68 = stock68.iloc[min_index_68]["time_id"]


In [ ]:
print("min index is",min_time_id_68,"min target is",stock68.iloc[min_index_68]["target"])
##

In [ ]:
stock68 = train[train["stock_id"]==68].reset_index()
max_index_68 = stock68["target"].idxmax()
max_time_id_68 = stock68.iloc[max_index_68]["time_id"]
print("max index is",max_time_id_68,"max target is",stock68.iloc[max_index_68]["target"])

In [ ]:
stock68

In [ ]:
book_example = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
trade_example =  pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')

In [ ]:
##
book_68 = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=68')
trade_68 =  pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=68')

In [ ]:
book_68
## 我其实没搞懂是这里选定一个time id还是前面那里选定time id？ 这里的time id有很多个，是需要也只选定一个吗？

In [ ]:
##
samples = ["bid_price1","bid_price2","ask_price1","ask_price2"]
book_test_min68 = book_68[book_68["time_id"]==min_time_id_68]
trade_test_min68 = trade_68[trade_68["time_id"]==min_time_id_68]


plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    
   
    plt.plot(book_test_min68["seconds_in_bucket"],book_test_min68[a],label=a)
    
plt.plot(trade_test_min68["seconds_in_bucket"],trade_test_min68["price"],label="trade_parquet",lw=10)
plt.legend(fontsize=12)

In [ ]:
book_test_max68 = book_68[book_68["time_id"]==max_time_id_68]
trade_test_max68 = trade_68[trade_68["time_id"]==max_time_id_68]


plt.figure(figsize=(20,5))

for num,a in enumerate(samples):
    
   
    plt.plot(book_test_max68["seconds_in_bucket"],book_test_max68[a],label=a)
    
plt.plot(trade_test_max68["seconds_in_bucket"],trade_test_max68["price"],label="trade_parquet",lw=10)
plt.legend(fontsize=12)
##

In [ ]:
##
for num,a in enumerate(samples):
    print(num,a)

In [ ]:
##
plt.figure(figsize=(20,5))
plt.plot(trade_test_min68["seconds_in_bucket"],trade_test_min68["price"],lw=10,label="min_vol_time")
plt.plot(trade_test_max68["seconds_in_bucket"],trade_test_max68["price"],lw=10,label = "max_vol_time")
plt.legend(fontsize=15)

In [ ]:
#When the volatility is high, in this case, it can be seen that the price fluctuates considerably in 10 minutes.

In [ ]:
##
stock_id = '68'
book_68 = book_68[book_68['time_id']==5]
book_68.loc[:,'stock_id'] = stock_id
trade_68 = trade_68[trade_68['time_id']==5]
trade_68.loc[:,'stock_id'] = stock_id

In [ ]:
book_68

In [ ]:
book_68.head()

In [ ]:
book_68.hist(figsize=(20,12))
plt.show()

In [ ]:
sns.pairplot(book_68[['seconds_in_bucket','ask_price1','ask_price2','ask_size1','ask_size2']],  diag_kind="hist",height=3)

In [ ]:
sns.pairplot(book_68[['seconds_in_bucket','bid_price1','bid_price2','bid_size1','bid_size2']],  diag_kind="hist",height=3)

In [ ]:
#seconds_in_bucket - Number of seconds from the start of the bucket, always starting from 0.
#bid_price[1/2] - Normalized prices of the most/second most competitive buy level.
#ask_price[1/2] - Normalized prices of the most/second most competitive sell level.
#bid_size[1/2] - The number of shares on the most/second most competitive buy level.
#ask_size[1/2] - The number of shares on the most/second most competitive sell level.

In [ ]:
trade_68.head()

In [ ]:
trade_68.hist(figsize=(20,12))
plt.show()

In [ ]:
sns.pairplot(trade_68[['seconds_in_bucket','price','size','order_count']],  diag_kind="hist",height=3)

In [ ]:
#price - The average price of executed transactions happening in one second. 
#Prices have been normalized and the average has been weighted by the number of shares traded in each transaction.
#size - The sum number of shares traded.
#order_count - The number of unique trade orders taking place.

In [ ]:
##
plt.figure(figsize = (10,8)) 
sns.lineplot(x = trade_68["seconds_in_bucket"], y =trade_68["size"], data = trade_68)
plt.legend(labels=['size'])
plt.show()

In [ ]:
plt.figure(figsize = (10,8)) 
sns.lineplot(x = trade_68["seconds_in_bucket"], y =trade_68["price"], data = trade_68)
plt.legend(labels=['price'])
plt.show()
##

In [ ]:
#As realized volatility is a statistical measure of price changes on a given stock, 
#to calculate the price change we first need to have a stock valuation at the fixed interval (1 second).
#We will use weighted averaged price, or WAP, of the order book data we provided.

In [ ]:
book_68['wap'] = (book_68['bid_price1'] * book_68['ask_size1'] +
                                book_68['ask_price1'] * book_68['bid_size1']) / (
                                       book_68['bid_size1']+ book_68['ask_size1'])

In [ ]:
book_68.head()

In [ ]:
fig = px.line(book_68, x="seconds_in_bucket", y="wap", title='WAP of stock_id_0, time_id_5')
fig.show()

In [ ]:
#To compute the log return, we can simply take the logarithm of the ratio between two consecutive WAP. 
#The first row will have an empty return as the previous book update is unknown, therefore the empty return data point will be dropped.

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

In [ ]:
book_68.loc[:,'log_return'] = log_return(book_68['wap'])
book_68 = book_68[~book_68['log_return'].isnull()]

In [ ]:
fig = px.line(book_68, x="seconds_in_bucket", y="log_return", title='Log return of stock_id_0, time_id_5')
fig.show()

In [ ]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
realized_vol = realized_volatility(book_68['log_return'])
print(f'Realized volatility for stock_id 68 on time_id 5 is {realized_vol}')

In [ ]:
#let's look at bid_price1, bid_price2

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["bid_price1"], 
        mode='lines', 
        name='bid_price1'
    )
)
fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["bid_price2"], 
        mode='lines', 
        name='bid_price2'
    )
)

fig.add_trace(
    go.Scatter(
        x=trade_68["seconds_in_bucket"], 
        y=trade_68["price"], 
        mode='lines', 
        name='trade_price'
    )
)

fig.show()

In [ ]:
#ask_price1, ask_price2.

In [ ]:
import cufflinks as cf
cf.go_offline()###这两句是离线生成图片的设置
cf.set_config_file(offline=True, world_readable=True)
### 如果下图go.Figure()显示失败，则需要调用这个block

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["ask_price1"], 
        mode='lines', 
        name='ask_price1'
    )
)
fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["ask_price2"], 
        mode='lines', 
        name='ask_price2'
    )
)

fig.add_trace(
    go.Scatter(
        x=trade_68["seconds_in_bucket"], 
        y=trade_68["price"], 
        mode='lines', 
        name='trade_price'
    )
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["bid_price1"], 
        mode='lines', 
        name='bid_price1'
    )
)
fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["bid_price2"], 
        mode='lines', 
        name='bid_price2'
    )
)

fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["ask_price1"], 
        mode='lines', 
        name='ask_price1'
    )
)
fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["ask_price2"], 
        mode='lines', 
        name='ask_price2'
    )
)

fig.add_trace(
    go.Scatter(
        x=trade_68["seconds_in_bucket"], 
        y=trade_68["price"], 
        mode='lines', 
        name='trade_price'
    )
)

fig.show()

In [ ]:
book_68.head()

In [ ]:
trade_68.head()

In [ ]:
temp_aggs = book_68.groupby(['time_id']).agg(
                                            bid_price1_min = ('bid_price1', 'min'),
                                            bid_price2_max = ('bid_price2', 'max'),
                                            bid_price1_mean = ('bid_price1', 'mean'),
                                            bid_price2_mean = ('bid_price2', 'mean'),
                                            bid_price1_median = ('bid_price1', 'median'),
                                            bid_price2_median = ('bid_price2', 'median'),
                                            ask_price1_min = ('ask_price1', 'min'),
                                            ask_price2_max = ('ask_price2', 'max'),
                                            ask_price1_mean = ('ask_price1', 'mean'),
                                            ask_price2_mean = ('ask_price2', 'mean'),
                                            ask_price1_median = ('ask_price1', 'median'),
                                            ask_price2_median = ('ask_price2', 'median'),
)

In [ ]:
temp_aggs

In [ ]:
aggs_book_68 = pd.merge(book_68, temp_aggs, on=['time_id'], how='left')
aggs_book_68

In [ ]:
plt.figure(figsize=(16, 8))

plt.plot(book_68['seconds_in_bucket'], book_68['bid_price1'])
plt.plot(book_68['seconds_in_bucket'], book_68['bid_price2'])

plt.plot(trade_68['seconds_in_bucket'], trade_68['price'])

plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['bid_price1_min'],
            s=150,
            label='bid_price1_min'
           )
plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['bid_price2_max'],
            s=150,
            label='bid_price2_max'
           )
plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['bid_price1_mean'],
            s=150,
            label='bid_price_mean1'
           )
plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['bid_price2_mean'],
            s=150,
            label='bid_price2_mean2'
           )
plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['bid_price1_median'],
            s=150,
            label='bid_price1_median1'
           )
plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['bid_price2_median'],
            s=150,
            label='bid_price2_median2'
           )


plt.xlabel('seconds_in_bucket', fontsize=12)
plt.ylabel('bid_price', fontsize=12)

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16, 8))

plt.plot(book_68['seconds_in_bucket'], book_68['ask_price1'])
plt.plot(book_68['seconds_in_bucket'], book_68['ask_price2'])

plt.plot(trade_68['seconds_in_bucket'], trade_68['price'])

plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['ask_price1_min'],
            s=150,
            label='ask_price1_min'
           )
plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['ask_price2_max'],
            s=150,
            label='ask_price2_max'
           )
plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['ask_price1_mean'],
            s=150,
            label='ask_price1_mean1'
           )
plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['ask_price2_mean'],
            s=150,
            label='ask_price2_mean2'
           )
plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['ask_price1_median'],
            s=150,
            label='ask_price1_median1'
           )
plt.scatter(x=aggs_book_68['seconds_in_bucket'], 
            y=aggs_book_68['ask_price2_median'],
            s=150,
            label='ask_price2_median2'
           )

plt.xlabel('seconds_in_bucket', fontsize=12)
plt.ylabel('ask_price', fontsize=12)

plt.legend()
plt.show()

In [ ]:
#bid_size and ask_size.

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["bid_size1"], 
        mode='lines', 
        name='bid_size1'
    )
)
fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["bid_size2"], 
        mode='lines', 
        name='bid_size2'
    )
)

fig.show()

In [ ]:
#bid_size1 is a little higher than bid_size2.

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["ask_size1"], 
        mode='lines', 
        name='ask_size1'
    )
)
fig.add_trace(
    go.Scatter(
        x=book_68["seconds_in_bucket"], 
        y=book_68["ask_size2"], 
        mode='lines', 
        name='ask_size2'
    )
)

fig.show()

In [ ]:
#ask_size1 is a little higher than ask_size2.
#In this case, the stock price is expected to rise because the buy level is higher than the sell level.
#It seems to be relatively high volatility.

In [ ]:
 temp_aggs = book_68.groupby(['time_id']).agg(
    bid_size1_min = ('bid_size1', 'min'),
    bid_size1_max = ('bid_size1', 'max'),
    bid_size2_min = ('bid_size2', 'min'),
    bid_size2_max = ('bid_size2', 'max'),
    bid_size1_mean = ('bid_size1', 'mean'),
    bid_size2_mean = ('bid_size2', 'mean'),
    bid_size1_median = ('bid_size1', 'median'),
    bid_size2_median = ('bid_size2', 'median'),
    
    ask_size1_min = ('ask_size1', 'min'),
    ask_size1_max = ('ask_size1', 'max'),
    ask_size2_min = ('ask_size2', 'min'),
    ask_size2_max = ('ask_size2', 'max'),
    ask_size1_mean = ('ask_size1', 'mean'),
    ask_size2_mean = ('ask_size2', 'mean'),
    ask_size1_median = ('ask_size1', 'median'),
    ask_size2_median = ('ask_size2', 'median'),              
)

In [ ]:
temp_aggs

In [ ]:
aggs_book_68_2 = pd.merge(aggs_book_68, temp_aggs, on=['time_id'], how='left')
aggs_book_68_2

In [ ]:
type(aggs_book_68_2['bid_size1_min'].min())

In [ ]:
plt.figure(figsize=(16, 8))

plt.scatter(x=1, 
            y=aggs_book_68_2['bid_size1_min'].min(),
            s=50,
            label='bid_size1_min1'
           )
plt.scatter(x=1, 
            y=aggs_book_68_2['bid_size1_max'].min(),
            s=aggs_book_68_2['bid_size1_max'].min() * 1.5,
            label='bid_size1_max1'
           )
plt.scatter(x=2, 
            y=aggs_book_68_2['bid_size2_min'].min(),
            s=50,
            label='bid_size2_min2'
           )
plt.scatter(x=2, 
            y=aggs_book_68_2['bid_size2_max'].min(),
            s=aggs_book_68_2['bid_size2_max'].min() * 1.5,
            label='bid_size2_max2'
           )
plt.scatter(x=1, 
            y=aggs_book_68_2['bid_size1_mean'].min(),
            s=50,
            label='bid_size1_mean1'
           )
plt.scatter(x=2, 
            y=aggs_book_68_2['bid_size2_mean'].min(),
            s=50,
            label='bid_size2_mean2'
           )
plt.scatter(x=1, 
            y=aggs_book_68_2['bid_size1_median'].min(),
            s=50,
            label='bid_size1_median1'
           )
plt.scatter(x=2, 
            y=aggs_book_68_2['bid_size2_median'].min(),
            s=50,
            label='bid_size2_median2'
           )


plt.xlabel('seconds_in_bucket', fontsize=12)
plt.ylabel('bid_size', fontsize=12)

plt.legend()
plt.show()

In [ ]:
#The difference between min and max for bid_size is large

In [ ]:
plt.figure(figsize=(16, 8))

# plt.plot(book_68['bid_price1'])
# plt.plot(book_68['bid_price2'])
plt.scatter(x=1, 
            y=aggs_book_68_2['ask_size1_min'].min(),
            s=50,
            label='ask_size1_min1'
           )
plt.scatter(x=1, 
            y=aggs_book_68_2['ask_size1_max'].min(),
            s=aggs_book_68_2['ask_size1_max'].min() * 1.5,
            label='ask_size1_max1'
           )
plt.scatter(x=2, 
            y=aggs_book_68_2['ask_size2_min'].min(),
            s=aggs_book_68_2['ask_size2_min'].min(),
            label='ask_size2_min2'
           )
plt.scatter(x=2, 
            y=aggs_book_68_2['ask_size2_max'].min(),
            s=aggs_book_68_2['ask_size2_max'].min() * 1.5,
            label='ask_size2_max2'
           )
plt.scatter(x=1, 
            y=aggs_book_68_2['ask_size1_mean'].min(),
            s=50,
            label='ask_size1_mean1'
           )
plt.scatter(x=2, 
            y=aggs_book_68_2['ask_size2_mean'].min(),
            s=50,
            label='ask_size2_mean2'
           )
plt.scatter(x=1, 
            y=aggs_book_68_2['ask_size1_median'].min(),
            s=50,
            label='ask_size1_median1'
           )
plt.scatter(x=2, 
            y=aggs_book_68_2['ask_size2_median'].min(),
            s=50,
            label='ask_size2_median2'
           )


plt.xlabel('size1 & size2', fontsize=12)
plt.ylabel('ask_size', fontsize=12)

plt.legend()
plt.show()

In [ ]:
#ask_size also has a big difference between min and max.

In [ ]:
# correlation
# order, trade, same stock and time id,merge, numeric feature correlation 
# seaborn heatmap

In [ ]:
##
booktrade=pd.merge(book_68, trade_68, on=['seconds_in_bucket'], how='left')

In [ ]:
booktrade

In [ ]:
# booktrade.fillna(0)

In [ ]:
sns.pairplot(booktrade[['seconds_in_bucket','price','size','order_count','bid_price1','bid_price2','bid_size1','bid_size2']],  diag_kind="hist",height=3)

In [ ]:
## price 与 bid_price 1 & bid_price 2 有较强的correlation